<a href="https://colab.research.google.com/github/UrienLin/CathayExam/blob/master/CrawlerSPEC_591.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as soup
import time
# install chromium, its driver, and selenium
!apt -qq install chromium-chromedriver 
!pip --quiet install selenium
# set options to be headless, ..
from selenium import webdriver


chromium-chromedriver is already the newest version (80.0.3987.87-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


取得列表內容[ID,TITLE,URL]

In [0]:
def extract_element(element):
  img = element.find_element_by_css_selector('img')
  href = element.find_element_by_css_selector('h3 a')
  
  ID = img.get_attribute('data-bind')
  title = img.get_attribute('title')
  url = href.get_attribute('href')
  return [ID,title,url]


由於網站爬取時需驗證COOKIES，雖然 NEW SESSION拿的到，XSRF-TOKEN無法從HTTP的方式取得
故REQUESTS無法模擬，只能用SELEIUM的方式取得



In [0]:
url = 'https://rent.591.com.tw/?kind=0&region=1&order=posttime&orderType=desc'
#for code in regions:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)


換頁相關 btn


# 翻頁功能
1.同 region 不同頁的切換，
如果“下一頁”按鈕有連結，就按該按鈕換頁，
如果該按鈕沒有連結，表示到了最後一頁。

2.由於直接改網址的 region 無法成功切換地區，
經測試須從選單點選 ‘新北市’ 後，
再對瀏覽器進行重新整理，方能成功切換地區。

In [46]:
#目前地區
current_url = driver.current_url
print('Current Url : {}'.format(current_url))
#目前頁碼
current_page = driver.find_element_by_css_selector('div.pageBar span.pageCurrent').get_attribute('innerText')
print('Current Page : {}'.format(current_page))
#下一頁按鈕
btn_next_page = driver.find_element_by_css_selector('div.pageBar a.pageNext')

#最後一頁的按鈕
btn_page_bar = driver.find_elements_by_css_selector('div.pageBar a.pageNum-form')
print('Page Bar Options : ',[x.get_attribute('innerText')for x in btn_page_bar])
btn_last_page = btn_page_bar[0] #firstPage = 0 , lastPage=-1

#等候顯示，按下按鈕
driver.execute_script("arguments[0].click();", btn_last_page)
print('\n ***** 跳至指定頁面, Wait for 5s... ***** \n')
time.sleep(5)


#沒有下一頁時，「最後一頁按鈕」 沒有href 
bool_next_page = driver.find_element_by_css_selector('div.pageBar a.pageNext').get_attribute('href')

print('got next page') if bool_next_page  else print('final page')

current_page = driver.find_element_by_css_selector('div.pageBar span.pageCurrent').get_attribute('innerText')
print('Current Page : {}'.format(current_page))



Current Url : https://rent.591.com.tw/?kind=0&order=posttime&orderType=desc&region=3
Current Page : 2
Page Bar Options :  ['1', '3', '4', '5', '6', '7', '317']

 ***** 跳至指定頁面, Wait for 5s... ***** 

got next page
Current Page : 1


In [12]:
#先列出瀏覽器目前網址
current_page = driver.current_url
print(current_page)
#按下 位置： 台北市 的按鈕 才會載入其他縣市列表

region_switch = driver.find_element_by_css_selector('span.search-location-span')
driver.execute_script("arguments[0].click();", region_switch)

#按下新北市之後 讓 rsList 讀取
new_taipei_city = driver.find_element_by_css_selector("ul li.city-li a[data-id='3']")
driver.execute_script("arguments[0].click();", new_taipei_city)

#對原本頁面重新整理
driver.refresh()

#檢查是否切換至新北市
current_page = driver.current_url
print(current_page)


https://rent.591.com.tw/?kind=0&order=posttime&orderType=desc&region=1
https://rent.591.com.tw/?kind=0&order=posttime&orderType=desc&region=3


In [0]:
url='https://m.591.com.tw/v2/rent?regionid=1'
headers= {'user-agent' :'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1'}
client = req.session()

resp = client.get(url, headers=headers)
#resp2 = req.get('https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=2&firstRow=60&totalRows=9432',
#                headers={'referer':'https://rent.591.com.tw/?kind=0&region=2'},
#                )

In [0]:
def extract_page_content()
  obj_extract = \
  [[x.select('img')[0]['data-bind'],
    x.select('h3 a')[0]['href'] , 
    x.select('img')[0]['title'],]
  for x in obj_list]
  _df = pd.DataFrame(obj_extract)
  _df.columns = ['id', 'url', 'title']
  _df.url = _df.url.apply(lambda x: 'http:' + x.replace('\\','/'))

  return _df

In [0]:
_df = pd.DataFrame(obj_extract)
_df.columns = ['id', 'url', 'title']
_df.url.apply(lambda x: 'http:' + x.replace('\\','/'))

0     http://rent.591.com.tw/rent-detail-4600321.html
1     http://rent.591.com.tw/rent-detail-8830565.html
2     http://rent.591.com.tw/rent-detail-8851662.html
3     http://rent.591.com.tw/rent-detail-8869640.html
4     http://rent.591.com.tw/rent-detail-8887309.html
5     http://rent.591.com.tw/rent-detail-8900775.html
6     http://rent.591.com.tw/rent-detail-8901879.html
7     http://rent.591.com.tw/rent-detail-8907934.html
8     http://rent.591.com.tw/rent-detail-8907939.html
9     http://rent.591.com.tw/rent-detail-8915509.html
10    http://rent.591.com.tw/rent-detail-8941113.html
11    http://rent.591.com.tw/rent-detail-8894515.html
12    http://rent.591.com.tw/rent-detail-8910279.html
13    http://rent.591.com.tw/rent-detail-2897519.html
14    http://rent.591.com.tw/rent-detail-8777253.html
15    http://rent.591.com.tw/rent-detail-8777256.html
16    http://rent.591.com.tw/rent-detail-8830750.html
17    http://rent.591.com.tw/rent-detail-8836348.html
18    http://rent.591.com.tw